In [1]:
import os
from pandas import DataFrame, read_csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Read in and organize relative data (dg over different time intervals)


In [2]:
#where do we live right now? Change path to locate files?
os.getcwd()

'c:\\Users\\ekahler\\Documents\\Python projects'

In [3]:
#Load the csv containing all relative differences (from a copy of what's in the archive)
file = r'Relative.csv'
df = pd.read_csv(file, usecols=['hub','station','2016','2018'])
#change usecols to specify which years to compare - cannot compare 2014, 2016, or 2018 with any other years 
#for Avra Valley stations, only for Tucson stations. Prior to 2014, meter D209 was used in Avra Valley. Here
#I only want to compare 2016 and 2018

In [4]:
#remove NAN's from dataframe and assign to new dataframe variable
newdf = df.dropna(thresh=4)
#reindex new dataframe so it doesn't get stuck when you loop over it in the future (gives key error otherwise)
newdf.index=range(len(newdf))

In [5]:
#upload the A10.parse results for TAMA that have been edited so all absolute values are present
file2 = r'TAMA_all.csv' #alternate format for doing this - convenient if path is different
ABS_all = pd.read_csv(file2, usecols=['Station Name', 'Date', 'Gravity'])
#upload the previously edited csv only containing absolute values for 2016 and 2018
file3 = r'Absolute1618.csv'
ABS = pd.read_csv(file3)

In [6]:
#add some blank columns to the dataframe, to be filled later
newdf = newdf.assign(ABS2016 = 0, ABS2018 = 0, ABSdiff = 0, dS = 0, Easting = 0, Northing = 0)

In [7]:
#loop over the absolute and relative dataframes and assign values to the absolute columns (sum of absolute value
#at hub plus the relative difference at the station)
for i in range(len(ABS)):
    for j in range(len(newdf)):
        if newdf['hub'][j] == ABS['station'][i]:
            newdf.loc[j,'ABS2016'] = ABS.loc[i,'2016']+ newdf.loc[j,'2016']
            newdf.loc[j,'ABS2018'] = ABS.loc[i,'2018']+newdf.loc[j,'2018']
for i in range(len(newdf)):
    newdf.loc[i,'ABSdiff'] = newdf.loc[i,'ABS2018'] - newdf.loc[i,'ABS2016'] #dg
    newdf.loc[i,'dS'] = newdf.loc[i,'ABSdiff']/12.77 #dS

In [8]:
#look ok?
newdf.head()

,hub,station,2016,2018,ABS2016,ABS2018,ABSdiff,Easting,Northing,dS
0,adtucs,a54,12223.822780,12228.788990,9.792524e+08,9.792524e+08,4.766210,0,0,0.373235
1,adtucs,aetucs,-13053.951170,-13039.606660,9.792271e+08,9.792271e+08,14.144510,0,0,1.107636
2,adtucs,c92,7238.217198,7221.092937,9.792474e+08,9.792474e+08,-17.324261,0,0,-1.356638
3,adtucs,fd62,16560.810550,16570.692340,9.792567e+08,9.792567e+08,9.681790,0,0,0.758167
4,adtucs,nn11,5833.728064,5836.330886,9.792460e+08,9.792460e+08,2.402822,0,0,0.188161


In [9]:
#remove rows of zeros (no abs diffs)
for i in range(len(newdf)):
    if newdf['ABSdiff'][i] == 0.0:
        newdf = newdf.drop(i)    
newdf.index=range(len(newdf)) #reorganize indeces again so future loops don't freak out        

In [10]:
#now add the absolute measurement dS values from the absolute column
#add rows that have 0's for column placeholders, to match the dataframe they are being attached to
for i in range(len(ABS)):
    newdf.loc[-1] = [0,ABS['station'][i],0,0,0,0,0,0,0, ABS['dS'][i]]
    newdf.index = newdf.index +1 #without this, it will replace a single row i times
newdf.index=range(len(newdf))#for the safety of future loops!

In [11]:
#add in columns of coordinates so that file can be uploaded to ArcMap
coords = pd.read_csv('TAMAcoords.csv')

In [12]:
#pull out coordinates from coords and match to appropriate lines in newdf
for j in range(len(coords)):
    for i in range(len(newdf)):
        if newdf['station'][i] == coords['Site_ID'][j]:
            newdf.loc[i,'Easting'] = coords.loc[j,'Easting']
            newdf.loc[i,'Northing'] = coords.loc[j,'Northing']
        

boo


In [13]:
#look good?
newdf.head()

,hub,station,2016,2018,ABS2016,ABS2018,ABSdiff,Easting,Northing,dS
0,adtucs,a54,12223.822780,12228.788990,9.792524e+08,9.792524e+08,4.766210,503293.0645,3571734.700,0.373235
1,adtucs,aetucs,-13053.951170,-13039.606660,9.792271e+08,9.792271e+08,14.144510,499613.0000,3564747.000,1.107636
2,adtucs,c92,7238.217198,7221.092937,9.792474e+08,9.792474e+08,-17.324261,501947.8100,3579884.060,-1.356638
3,adtucs,fd62,16560.810550,16570.692340,9.792567e+08,9.792567e+08,9.681790,494738.0000,3573014.000,0.758167
4,adtucs,nn11,5833.728064,5836.330886,9.792460e+08,9.792460e+08,2.402822,494114.7760,3568134.478,0.188161


In [14]:
#kick out the outliers
limit = 10 #set limit beyond which dS is probably just bad
for i in range(len(newdf)):
    if newdf['dS'][i] > limit:
        newdf = newdf.drop(i)
newdf.index=range(len(newdf)) #!

In [15]:
#check to make sure every line has coordinates. If not, add to coordinate CSV and rerun
for i in range(len(newdf)):
    if newdf['Easting'][i] == 0:
        print(i)
        print(newdf['station'][i])

In [16]:
#create a new dataframe that is a subset of the columns in the main dataframe
outdf = newdf[['station','Easting','Northing','dS']]

In [17]:
#check to make sure all absolute (hub) stations are represented, and check to see how different
#the dS values are when they are present multiple times (from a relative survey and the absolute differences)
for i in range(len(outdf)):
    for j in range(len(ABS)):
        if outdf['station'][i] == ABS['station'][j]:
            outdf.loc[i,'dS'] = ABS.loc[j,'dS']
            print([outdf['station'][i],outdf['dS'][i]])

['aetucs', 0.187940484]
['c92', -0.344557564]
['fd62', 0.563821451]


C:\Users\ekahler\AppData\Local\Continuum\miniconda3\envs\pyg35\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


['c92', -0.344557564]
['aatucs', 1.158966324]
['af14', 1.119812065]
['aetucs', 0.187940484]
['av25', 1.127642911]
['af14', 1.119812065]
['av25', 1.127642911]
['c22a', 0.6577916970000001]
['c92', -0.344557564]
['enrb', -0.015661711000000002]
['fd62', 0.563821451]
['aatucs', 1.158966324]
['cetucs', 0.187940484]
['cftucs', 0.078308536]
['wr175a', -0.501174636]
['adtucs', -0.015661711000000002]
['aetucs', 0.187940484]


In [18]:
#output a csv file for uploading to ArcMap
#Note: May have to open this file and delete index column or save as something else
#for ArcMap to recognize the correct fields when projecting, but shouldn't have to, 
#it's just an ArcMap bug.
outdf.to_csv('dSforKriging.csv')